In [2]:
import sh
import json
import os
import sys
import re
import io
import pandas as pd
import subprocess

In [3]:
#LUCENE-SOLR
project_folder = "lucene"
output_folder = "out"

repository_folder = "lucene-solr"
repository_link = "https://github.com/apache/lucene-solr.git"
checkout_branch = "trunk"

issues_folders = ["issue_LUCENE", "issue_SOLR"]
issues_tags = ["LUCENE", "SOLR"]

In [4]:
#CAMEL
project_folder = "camel"
output_folder = "out"

repository_folder = "camel"
repository_link = "https://github.com/apache/camel"
checkout_branch = "master"

issues_folders = ["issue_CAMEL"]
issues_tags = ["CAMEL"]

In [5]:
def print_progress(i, l, step=1):
    if i % step == 0:
            print("\r>> {}/{}".format(i + 1, l), end="")
            sys.stdout.flush()

In [6]:
if not os.path.exists(os.getcwd() + '/' + project_folder + '/' + repository_folder):
    print("Path doesn\'t exists, cloning repo:")
    git = sh.git.bake(_cwd=project_folder)
    git.clone(repository_link)
else:
    print("Repository exists")

Repository exists


In [7]:
git = sh.git.bake(_cwd=project_folder + '/' + repository_folder)
git.checkout(checkout_branch)
git.status()

On branch master
Your branch is up-to-date with 'origin/master'.

nothing to commit, working directory clean

In [8]:
def extract_bug(file_path):
    if not file.endswith('.json'):
            return None
    else:
        bug_json_string = open(file_path).read()
        bug = json.loads(bug_json_string)
        bug_fields = bug.get('fields')
        
        if bug_fields['issuetype']['name'] != 'Bug':
            return None
   
        if bug_fields['resolution'] == None:
            return None 
                
        if bug_fields['resolution']['name'] != 'Fixed':
            return None
    return bug

In [9]:
#Extracting all the fixed bugs from the bug repo
bugs = dict() #Dictionary of bugs indexed by TAG+ID (e.g. LUCENE-1234)
for fld in issues_folders:
    path = project_folder + '/' + fld
    dir_list = os.listdir(os.getcwd() + '/' + path)
    l = len(dir_list)
    print()
    print("Progress for " + fld + ":")
    for idx, file in enumerate(dir_list):
        print_progress(idx,l,10)
        bug = extract_bug(path + '/' + file)
        if(bug == None):
            continue
        bugs[bug['key'].upper()] = bug

print()
print(len(bugs), "fixed bugs extracted.")


Progress for issue_CAMEL:
>> 9001/9002
2525 fixed bugs extracted.


In [10]:
commits = []

log = git.log("--reverse", "--pretty=format:%H")
for c_hash in log:
    if(c_hash.endswith('\n')):
        c_hash = c_hash[:-1]
    commits.append(c_hash)
len(commits)

21942

In [11]:
def get_fixed_bugs(commit_hash):
    details = git.show("--name-only","--pretty=format:%B", commit_hash)
    #init = details.split(':')[0].upper()
    f_keys = []
    
    for tag in issues_tags:
        keys = re.findall(tag + "-[0-9]+",str(details))
        if len(keys) != 0:
            for k in keys:
                if bugs.get(k) != None:
                    #Then it fixes a bug
                    f_keys.append(k)
    return f_keys

In [20]:
#Splits the bug fixing ones from the others

fix_commits = [] #List that will be filled with couples (commit, bugs-fixed)
non_fix_commits = [] #List that will be filled only with commit hashes
l = len(commits)
print("Progress:")
for idx, c in enumerate(commits):
    
    print_progress(idx, l, 10)
        
    details = git.show("--name-only","--pretty=format:%B", c)
    files = git.show("--name-only","--pretty=format:",c)
    
    
    f_keys = get_fixed_bugs(c)    
    if len(f_keys) != 0:
        fix_commits.append((c,f_keys))
    else:
        non_fix_commits.append(c)
        
#Some commits cite more than one issue...which lines are the bug fix, if one of the two is only an improvement ?
#HP: we consider all the removed lines as implicated (fix-inducing)

Progress:
>> 391/21942

In [21]:
print(len(fix_commits)) #Number of LUCENE bug fixing commits
print(len(non_fix_commits)) #Number of LUCENE non bug fixing commits
print(len(commits)) #Number of LUCENE + SOLR commits
#We can see that on average half of the commits were only solr related, so we discarded them

65
335
21942


In [22]:
def get_added_lines(commitHash):
    
    # Get diff of current and previous commit and store this in a file
    with open(project_folder + '_diff_file.txt', 'wb+') as file:
        cmd = ["git " + "-C ./" + project_folder + '/' + repository_folder 
               + '/' + " diff --no-color " + commitHash +'^ ' + commitHash]
        gitDiff = subprocess.Popen(cmd, shell=True, universal_newlines=True, stdout=file)
        gitDiff.wait()
        file.flush()

    # Get diff of the commit
    output_diff = ""
    with open(project_folder + '_diff_file.txt', 'rb+') as file:
        output_diff = file.read()
    
    file.close()
    
    # Cut output up in separate lines
    lines = output_diff.decode("latin1").split('\n')
    
    line_num = dict()
    start_line = 0
    name_of_file = ""
    i = 0
    name_file_set = set()

    # Go through the diff output to determine line number(s) of deleted lines
    for line in lines:
        # Header where the name of the file is located.
        if line[0:4] == "diff":
            name_of_file = line.split(' b/')[-1]


        # New section of changed code find starting line number
        elif line[0:2] == "@@":
            lineNum = re.findall('\-[0-9]+\,',line)
            start_line = int(re.findall('\-[0-9]+',line)[0][1:])
            i = 0

        # Lines that has been changed
        elif (line[0:2] == "- "):
            if line_num.get(name_of_file) == None:
                # create set of lines that have been changed for each file
                line_num[name_of_file] = []
                name_file_set.add(name_of_file)
            # Add lines to the set that have been changed
            line_num[name_of_file].append(start_line + i);
            i += 1


        # Indentaion infront of a line of code that hasn't changed
        elif line[0:2] == "  ":
            i += 1
            
    return line_num

In [23]:
def get_bug_commits(commit_hash, line_num, bug_hash):
    # checkout previous commit
    git.checkout("--force", commit_hash + "^")

    # Go through all the files that have been changed
    for key_file_name in line_num:
        ## Check if file is a java file if not continue with next file
        if(key_file_name.find('.java') == -1):
            continue
        
        # blame file to see when the lines where last changed (on previous commit)
        with open(project_folder + '_blame_file.txt', 'wb+') as file:
            # --root is added to see boundary commits as normal commits
            cmd = ["git " + "-C ./"+ project_folder + '/' + repository_folder + '/' +" blame -l --root " + key_file_name]
            gitBlame = subprocess.Popen(cmd, shell=True, universal_newlines=True, stdout=file)
            gitBlame.wait()
            file.flush()
            
        output_blame = ""
        with open(project_folder + '_blame_file.txt', 'rb+') as file:
            output_blame = file.read()

        # Cut output up in separate lines
        blame_output = output_blame.decode("latin1")        
        blame_output_split = blame_output.split('\n')
                
        # Get hash for all the lines that have been changed.
        for i in line_num[key_file_name]:
            line = blame_output_split[i-1]
            hash_of_line = line.split(' ')[0]
            
            if bug_hash.get(hash_of_line) == None:
                bug_hash[hash_of_line] = set([key_file_name])
            else:
                bug_hash[hash_of_line].add(key_file_name)
            
    return bug_hash

In [24]:
total = len(fix_commits)
bug_commits = dict()

print("Progress:")
for idx, fix_hash in enumerate(fix_commits):
    new_lines = get_added_lines(fix_hash[0])
    bug_commits = get_bug_commits(fix_hash[0], new_lines, bug_commits)

    print_progress(idx, total,1)

Progress:
>> 64/65

In [25]:
hash_file_list = []
for key, file_name_set in bug_commits.items():
    for file_name in file_name_set:
        hash_file_list.append([key, file_name, 1])

In [27]:
df = pd.DataFrame(hash_file_list, columns=['sha','file', 'implicated'])
df.to_csv(output_folder + '/' +  project_folder + '_implicated_files.csv', index = False)

df


,sha,file,implicated
0,a54c05c94e2a26b2e58b3223d98abdb4dc73a782,camel-core/src/main/java/org/apache/camel/impl...,1
1,0ea40fae717241f5322d95b60233f982ae810040,camel-core/src/main/java/org/apache/camel/proc...,1
2,661a7921f2867927b54ad46cc07a7c4b5ee4e5f0,camel-core/src/main/java/org/apache/camel/impl...,1
3,93e80381178d5707721a64dc5f1f9e3602d937ff,components/camel-jetty/src/main/java/org/apach...,1
4,93e80381178d5707721a64dc5f1f9e3602d937ff,components/camel-jetty/src/main/java/org/apach...,1
5,6215ed0165030fe3e44670d490158b2c476f278e,components/camel-cxf/src/main/java/org/apache/...,1
6,6215ed0165030fe3e44670d490158b2c476f278e,components/camel-cxf/src/main/java/org/apache/...,1
7,412202619255dea906c6623dd4138d507a5a3fd6,components/camel-spring/src/main/java/org/apac...,1
8,5112cf705cfd05eb46b49e3c6d8f6adefea48b9f,camel-core/src/main/java/org/apache/camel/proc...,1
9,98ef5e9959410360c01f145e100a71fb52f23b2d,components/camel-cxf/src/main/java/org/apache/...,1
